# Web Scraping and Geocoding With Python
This is just a fun side project to demonstrate web scraping basics and geocoding. 

The first part of the application starts with a basic list of companies, goes out to [Google Finance](https://www.google.com/finance) and pulls down address information in addition to other key financial information.

Once all of the key addresses have been loaded, the next step is to bounce those addresses against [Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/intro) to pull in Latitude and Longitude. 

The last part is to take that data, create a basic scatter plot in Bokeh, and then overlays that plot on top of a Google Map.

## Key Libraries Used
+ [Pandas](http://pandas.pydata.org/)
+ [Requests](http://docs.python-requests.org/en/master/)
+ [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
+ [Bokeh](http://bokeh.pydata.org/en/latest/)

----

Library import for web scraping and geocoding

In [1]:
import pandas as pd
from lxml import html
import requests
import bs4
from collections import OrderedDict
from copy import deepcopy

Import a previously constructed list of companies, names only, into a Pandas dataframe.

In [2]:
df_company = pd.read_csv("data/companylist.csv")

Next is just a quick check to validate the first few records were loaded succesfully.

In [3]:
df_company.head(5)

,CompanyName
0,ACUITY Insurance
1,ADBE
2,ADSK
3,"ARI, Automotive Resources International"
4,ATVI


Since this is more of a *row based* operation rather than *set based*, the dataframe is converted into a list. There are more effecient ways to to this, but for the moment "practicality beats purity...".

In [4]:
companylist = sorted(set(df_company["CompanyName"].tolist()))

Since there is no guaranteed list of fields to be returned, some values may be empty, or not even present on the data returend from the web page.

Therefore, a basic list of fields is created, and populated when present. If not, the value will remain empty. This allows for the easy construction back into a dataframe later.

In [5]:
#base field set
fields = ["companyName"
          ,"exchange"
          ,"symbol"
          ,"companySummary"
          ,"management"
          ,"website"
          ,"address"
          ,"city"
          ,"state"
          ,"zip"
          ,"lat"
          ,"lng"]

The primary loop below does the majority of the work. Since it's web scraping and passing data back and forth to external URls, it can take time. Please see comments for details.

In [6]:
companydata = [] #List of dictionary items

for index, item in enumerate(companylist):
    
    #Printing if only to know where in the process the system.
    print("Searching for - ",item)
    
    #Dynamically created URL based on company name
    import_url = "https://www.google.com/finance?q=%s" % item 
    
    #URL used for geo coding
    geo_url = "https://maps.googleapis.com/maps/api/geocode/json"
    
    #response header to grab data
    response = requests.get(import_url)
    
    #interprets XML/HTML using the LXML toolkit
    soup = bs4.BeautifulSoup(response.text,"lxml")
    
    #empty dictionary to store 
    companydict = {}
    
    #Basic variables for temp storage
    companyName = ""
    exchange = ""
    symbol = ""
    companySummary = "" 
    companydict['lat'] = ""
    companydict['lng'] = ""
    full_address = ""
    
    #grabs the page header
    header = soup.title.text.split(":")

    #base attributes
    if len(header) >= 1:
        companyName = header[0].rstrip()                
    if len(header) >= 2:      
        exchange = header[1].rstrip()
    if len(header) >= 3:      
        symbol = header[2].replace('\n', '').split(" ")[0].rstrip()

    #company summary
    #Private companies only have a basic summary, but this can still capture that
    companySummarySelect = soup.select('.companySummary')
    if len(companySummarySelect) >= 1:
        companySummary = companySummarySelect[0].text.rstrip()    
        companySummary = companySummary.replace('More from Reuters »', '')
        companySummary = companySummary.replace('\n', ' ')
        companySummary = companySummary.replace('\r', '')
        companySummary = companySummary.strip().rstrip()
    
    #Validate company exists on public exchange or there is at least a summary
    if exchange == "NASDAQ" or exchange == "NYSE" or len(companySummary) > 0:        
        
        #print("    Found - ",companyName)

        companydict['companyName'] = companyName
        companydict['exchange'] = exchange
        companydict['symbol'] = symbol
        #companydict['companySummary'] = companySummary
        
        #Address 
        #Looking at patterns within specific tag structures to pull out address information
        address_text = soup.find(text="Address")
        if address_text != None:
            address_div = address_text.parent.findNext('div')
            if address_div != None:
                address_contents = address_div.contents
                for idx,item in enumerate(address_contents):
                    if idx == 0:
                        companydict['address'] = item.rstrip()
                    if idx == 2:
                        companydict['city'] = item.split(",")[0].rstrip()
                        if len(item.split(",")) >= 2:
                            companydict['state'] = item.split(",")[1].split(" ")[1].rstrip()
                            if len(item.split(",")[1].split(" ")) >= 3:
                                companydict['zip'] = item.split(",")[1].split(" ")[2].rstrip()
        
       
        
        #Geocode valid address
        if len(companydict['address']) > 0 and set(['city','state','zip']).issubset(set(companydict.keys())):
            
            full_address = companydict['address'] + ' ' + companydict['city'] + ', ' + companydict['state'] + ' ' +companydict['zip']
            #print("Geocoding ",full_address)                       
            
            params = {'sensor': 'false', 'address': full_address}            
            r = requests.get(geo_url, params=params)
            results = r.json()['results']
            if len(results) > 0:
                location = results[0]['geometry']['location']                
                companydict['lat'] = location['lat']
                companydict['lng'] = location['lng']
                #print(location['lat'],location['lng'])
            

        #website
        webtext = soup.find(text="Website links") 
        if webtext != None:
            url_link = soup.find(text="Website links").parent.findNext('a')
            if  url_link != None:
                website = url_link.text.rstrip()                
                website = website.replace('\n', ' ')
                website = website.replace('\r', '')
                website = website.strip().rstrip()
                
                companydict['website']  = website
        
        #key stats
        #EBITD / Net Profit Margin / etc.
        market_list = soup.select('.lft')
        for idx,item in enumerate(market_list):      
            if len(item.text.rstrip()) > 0:
                fields.append(item.text.rstrip())
                companydict[item.text.rstrip()] = item.findNext('td').text.rstrip()

        #current pricing info (EPS/Range/52 Week High)
        attrib_list = soup.select(".key")
        for idx, item in enumerate(attrib_list):  
            if len(item.text.rstrip()) > 0:
                fields.append(item.text.rstrip())
                companydict[item.text.rstrip()] = item.findNext('td').text.rstrip()
                
        #mgmt-section        
        mgmt_section = soup.select('.id-mgmt-table')
        
        if len(mgmt_section) > 0:
            mgmt = mgmt_section[0].findAll('tr')

            management = []
            for idx,tblrow in enumerate(mgmt):
                if "style" not in tblrow.attrs:        
                    rowdata = tblrow.findAll('td')
                    cexec = ""
                    ctitle = ""
                    for idx,tbldata in enumerate(rowdata):           
                        if "class" in tbldata.attrs:         
                            if "linkbtn" in tbldata.attrs["class"]:
                                cexec = tbldata.text.rstrip()
                            if "t" in tbldata.attrs["class"]:
                                ctitle = tbldata.text.rstrip()
                    if len(cexec) > 0:
                        management.append(cexec + ":" + ctitle)
            
            #This is left out for now; future goal is to create a graph database of Corporate Interlocks
                        
            if len(management) > 0:            
                #companydict["management"] = management
                pass
        
        #Add item to list
        companydata.append(companydict)
            
    else:
        #print("    ",item," - not found")
        companydict['companyName'] = item

###  Align and populate

Since not all data returned has all fields populated, or even in the same order, this applies a list of fields to all 
values, in order. If the field is populated, it updates the list. This ensures that the final set is aligned for output in the dataframe. 

Here, deecopy() is used to copy empty fields references which are then populated as available. 

Looking at the official Python [documentation](https://docs.python.org/3/library/copy.html) 
The difference between shallow and deep copying is only relevant for compound objects (objects that contain other objects, like lists or class instances):

+ A shallow copy constructs a new compound object and then (to the extent possible) inserts references into it to the objects found in the original.
+ A deep copy constructs a new compound object and then, recursively, inserts copies into it of the objects found in the original.

In [7]:
#Create distinct, sorted list of all fields pulled
field_set = sorted(set(fields))
companydata_sorted = []

#remove empty field
if '' in field_set:
    field_set.remove('')

#creates ordered dict
field_dict = OrderedDict()

#emptys ordered dict
for idx,field in enumerate(field_set):    
    field_dict[field] = ""

for idx,company_data in enumerate(companydata):
    #print(company_data['company'])
    tmp_dict = deepcopy(field_dict)   
    for idx,field in enumerate(company_data):
        if field in tmp_dict:
            tmp_dict[field] = company_data[field]
            
    companydata_sorted.append(tmp_dict)  

In [8]:
#print(companydata_sorted)
df = pd.DataFrame(companydata_sorted)

In [9]:
df_valid = df[(df['lat'].str.contains("")!=True) & (df['lng'].str.contains("")!=True)]

In [10]:
#use variables here to easily change as needed
max_lng = -120
min_lng = -123
max_lat = 39.0
min_lat = 37.0

#only companies in SF Bay Area
df_sf = df_valid[(df_valid['lat']<=max_lat) 
                 & (df_valid['lat']>=min_lat) 
                 & (df_valid['lng']<=max_lng) 
                 & (df_valid['lng']>=min_lng)]

In [11]:
df_sf[['companyName','address','city','state','zip','lat','lng']].head(10)

,companyName,address,city,state,zip,lat,lng
0,Adobe Systems Incorporated,345 PARK AVE,SAN JOSE,CA,95110-2704,37.3307,-121.894
1,"Autodesk, Inc.",111 MCINNIS PKWY,SAN RAFAEL,CA,94903,38.0055,-122.531
4,"Advanced Micro Devices, Inc.",1 Amd Pl MS 68,SUNNYVALE,CA,94085-3905,37.3895,-122.018
6,Agilent Technologies Inc,5301 Stevens Creek Blvd,SANTA CLARA,CA,95051-7201,37.3249,-121.999
9,Apple Inc.,1 Infinite Loop,CUPERTINO,CA,95014-2083,37.3317,-122.03
12,"Brocade Communications Systems, Inc.",130 Holger Way,SAN JOSE,CA,95134,37.4185,-121.952
19,"Cisco Systems, Inc.",170 W Tasman Dr,SAN JOSE,CA,95134-1706,37.4084,-121.954
21,Chevron Corporation,6001 Bollinger Canyon Rd,SAN RAMON,CA,94583-2324,37.7583,-121.958
29,Cooley LLP,3175 Hanover St,Palo Alto,CA,94304-1130,37.4146,-122.142
34,Electronic Arts Inc.,209 Redwood Shores Pkwy,REDWOOD CITY,CA,94065-1175,37.5233,-122.254


### Import Bokeh library and models

In [13]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (
  GMapPlot, GMapOptions, ColumnDataSource, Circle, DataRange1d, PanTool, WheelZoomTool, BoxSelectTool, HoverTool, ResetTool
)

### Output Bokeh to the notebook
This tells Bokeh to output to the notebook rather than to an HTML file.

In [14]:
output_notebook()

Loading BokehJS ...

In [30]:
source = ColumnDataSource(
    data=dict(
        lat = df_sf['lat'].tolist(), 
        lng = df_sf['lng'].tolist(),  
        company = df_sf['companyName'].tolist(),  
    )
)

hover = HoverTool(
        tooltips=[          
           ("Company ", "@company"),
        ]
    )

p = figure(plot_width=600, plot_height=400, tools=[hover,PanTool()],
           title="Scatter Plot of geographical locations")

p.circle('lat', 'lng', size=10, source=source)

### Show the plot

In [31]:
show(p)

In [37]:
# satellite, roadmap, terrain or hybrid
# Create initial map and center it over SF Bay area
map_options = GMapOptions(lat=37.4224497, lng=-122.0840329, map_type="roadmap", zoom=9)

source = ColumnDataSource(
    data=dict(       
        lat = df_sf['lat'].tolist(), 
        lng = df_sf['lng'].tolist(),          
        company = df_sf['companyName'].tolist(),  
    )
)

hover = HoverTool(
        tooltips=[                       
            ("Company ", "@company"),
        ]
    )

plot = GMapPlot(
    x_range=DataRange1d(), y_range=DataRange1d(), map_options=map_options, title="Bay Area Companies"
    ,plot_width=800,plot_height=600
)

circle = Circle(x="lng", y="lat", size=15, fill_color="blue", fill_alpha=0.8, line_color=None)
plot.add_glyph(source, circle)

plot.add_tools(PanTool(), WheelZoomTool(), BoxSelectTool(),ResetTool(),hover)

In [38]:
show(plot)